In [1]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('factbook.db')
info = 'SELECT * FROM sqlite_master WHERE type="table";'
pd.read_sql_query(info, conn)


,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY..."


In [3]:
q_select_all = 'SELECT * FROM facts'
first_five_facts = pd.read_sql_query(q_select_all, conn)[0:6]
first_five_facts

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
0,1,af,Afghanistan,652230.0,652230.0,0.0,32564342.0,2.32,38.57,13.89,1.51
1,2,al,Albania,28748.0,27398.0,1350.0,3029278.0,0.30,12.92,6.58,3.30
2,3,ag,Algeria,2381741.0,2381741.0,0.0,39542166.0,1.84,23.67,4.31,0.92
3,4,an,Andorra,468.0,468.0,0.0,85580.0,0.12,8.13,6.96,0.00
4,5,ao,Angola,1246700.0,1246700.0,0.0,19625353.0,2.78,38.78,11.49,0.46
5,6,ac,Antigua and Barbuda,442.0,442.0,0.0,92436.0,1.24,15.85,5.69,2.21


In [6]:
q_min_max_pop ='SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth) FROM facts'
pd.read_sql_query(q_min_max_pop, conn)

,MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,0,7256490011,0.0,4.02


In [7]:
q_min_pop_name = 'SELECT name, population, population_growth FROM facts WHERE population = 0 OR population_growth = 0'
pd.read_sql_query(q_min_pop_name, conn)

,name,population,population_growth
0,Holy See (Vatican City),842,0.0
1,Cocos (Keeling) Islands,596,0.0
2,Greenland,57733,0.0
3,Pitcairn Islands,48,0.0
4,Antarctica,0,NaN


In [8]:
q_max_pop_name = 'SELECT name, population, population_growth FROM facts WHERE population = 7256490011'
pd.read_sql_query(q_max_pop_name, conn)

,name,population,population_growth
0,World,7256490011,1.08


In [9]:
q_max_popgrow_name = 'SELECT name, population, population_growth FROM facts WHERE population_growth = (SELECT MAX(population_growth) FROM facts)'
pd.read_sql_query(q_max_popgrow_name, conn)

,name,population,population_growth
0,South Sudan,12042910,4.02


## Observations

So basically, the Factbook contains information on not just countries, but landmasses, including `Antarctica` and `World`. I wonder if they include regions... I'm going to investigate.

In [10]:
q_gt_onebillion = 'SELECT * FROM facts WHERE population > 1000000000'
pd.read_sql_query(q_gt_onebillion, conn)

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
0,37,ch,China,9596960.0,9326410.0,270550.0,1367485388,0.45,12.49,7.53,0.44
1,77,in,India,3287263.0,2973193.0,314070.0,1251695584,1.22,19.55,7.32,0.04
2,261,xx,World,NaN,NaN,NaN,7256490011,1.08,18.60,7.80,NaN


## No continents/regions

Based on the `q_gt_onebillion` query, there are no regions or continents included in the factbook, only `World` and `Antarctica` are outliers. The remaining entries in the `facts` table should all be countries. I doubled checked this by looking up the population of [Africa](https://www.google.com/search?q=africa+population&oq=africa+population&aqs=chrome.0.0l6.2259j0j7&sourceid=chrome&ie=UTF-8) which is 1.26 billion and would have appeared in the query if it was present.

I'll have to take these two entries into account when dealing with the database. Or just remove them from the dataset. Probably by making a copy of the data and excluding those two entries. 

In [1]:
import pandas as pd
import sqlite3
conn = sqlite3.connect('factbook.db')

q_wo_ant_world = 'SELECT population, population_growth, birth_rate, death_rate FROM facts WHERE population > 0 AND population < 2000000000 ORDER BY population'
data_wo_outliers = pd.read_sql_query(q_wo_ant_world, conn)
data_wo_outliers[0:10]


,population,population_growth,birth_rate,death_rate
0,48,0.00,NaN,NaN
1,596,0.00,NaN,NaN
2,842,0.00,NaN,NaN
3,1190,0.03,NaN,NaN
4,1337,0.01,NaN,NaN
5,1530,1.11,NaN,NaN
6,1872,0.03,NaN,NaN
7,2210,0.01,NaN,NaN
8,3361,0.01,10.90,4.9
9,5241,0.50,11.26,6.3


## Next Steps

Going to skip the plotting stuff for now, just practice the querying aspect.

Questions to explore:

1. Which countries have the highest population density?
    - Population density is the ratio of population to land area. 
    - Compare with [Wikipedia](https://simple.wikipedia.org/wiki/List_of_countries_by_population_density)
2. Which countries have the highest ratios of water to land?
3. Which countries have more water than land?

### Population Density

In [3]:
q_pop_density = 'SELECT name, CAST(population as float) / CAST(area_land as float) as population_density FROM facts WHERE population != (SELECT MIN(population) FROM facts) AND population != (SELECT MAX(population) FROM facts) ORDER BY population_density DESC LIMIT 10'
pd.read_sql_query(q_pop_density, conn)

,name,population_density
0,Macau,21168.964286
1,Monaco,15267.500000
2,Singapore,8259.784571
3,Hong Kong,6655.271202
4,Gaza Strip,5191.819444
5,Gibraltar,4876.333333
6,Bahrain,1771.859211
7,Maldives,1319.640940
8,Malta,1310.015823
9,Bermuda,1299.925926


## Wikipedia
1. Macau
2. Monaco
3. Singapore
4. Hong Kong
5. Gibralter
6. Vatican City
7. Bahrain
8. Malta
9. Bermuda
10. Sint Maarten (Netherlands)

Very close. The addition of the Gaza strip is altering the list. And for some reason the Vatican does not feature. I'm going to look into it:

In [4]:
q_vatican = 'SELECT name, population, area_land FROM facts WHERE name = "Holy See (Vatican City)"'
pd.read_sql_query(q_vatican, conn)

,name,population,area_land
0,Holy See (Vatican City),842,0


As `area_land` is recorded as 0 (I imagine this is due to the miniscule size of the state, which is recorded as only [110 acres](https://simple.wikipedia.org/wiki/Vatican_City); the `area_land` column probably requires an `integer`), the Vatican was not included in `population_density`. It would have raised a zero-division error.

### Highest Ratio Water to Land

In [6]:
q_water_ratio = 'SELECT name, CAST(area_water as float)/CAST(area_land as float) as water_ratio FROM facts WHERE population != (SELECT MIN(population) FROM facts) AND population != (SELECT MAX(population) FROM facts) ORDER BY water_ratio DESC LIMIT 10'
pd.read_sql_query(q_water_ratio, conn)

,name,water_ratio
0,Virgin Islands,4.520231
1,Puerto Rico,0.554791
2,"Bahamas, The",0.386613
3,Guinea-Bissau,0.284673
4,Malawi,0.259396
5,Netherlands,0.225710
6,Uganda,0.222922
7,Eritrea,0.164356
8,Liberia,0.156240
9,Bangladesh,0.140509


### More Water than Land

In [7]:
q_water_gt_land = 'SELECT name, CAST(area_water as float)/CAST(area_land as float) as water_ratio, area_water, area_land FROM facts WHERE area_water > area_land'
pd.read_sql_query(q_water_gt_land, conn)

,name,water_ratio,area_water,area_land
0,British Indian Ocean Territory,905.666667,54340,60
1,Virgin Islands,4.520231,1564,346
